# Pipeline de experimentos usando classificadores (o atributo alvo é categórico)
Os resultados serão obtidos por meio de validação cruzada.

# PARAMETROS
Seleção da coluna alvo e da quantidade de subconjuntos (folds) para a validação cruzada.

In [1]:
# Coluna alvo, por exemplo: DDF	Ca	Ce	Ch	Am	Ma	ADF	meso
TARGET = 'FUNÇÃO'
# Quantidade de subconjuntos para a validação cruzada
FOLDS = 6

# Atributos a serem selecionados pelo seletor de atributos KBEST
ATRIBUTOS_KBEST = 3
# Número de dimensões a serem utilizadas pelo redutor de dimensionalidade
DIMENSOES_PCA = 2

## Conjunto de Dados
Aqui os dados são carregados e os valores faltantes são substituídos por zero.

In [2]:
from google.colab import drive

import pandas as pd
import numpy as np
#data = pd.read_csv("https://www.each.usp.br/digiampietri/dados/Cereeae_traits3.csv", sep='\t', encoding = 'UTF-8')
#data = pd.read_csv("http://143.107.58.250/temp/cibercrimes.csv", sep='\t', encoding = 'UTF-8')

drive.mount('/content/drive')
#new_data = "/content/metricas_locais_100_v2.txt"
new_data = "./drive/MyDrive/Análise de redes sociais criminais/metricas_locais_100_v3.txt"

data = pd.read_csv(new_data, sep="\t")

# Substituindo valores faltantes / not a numbers por zero
data.fillna(0)
data = data.replace(np.nan, 0)

data = data[(data['FUNÇÃO'] != 'não sei')]

# Exibindo parte do conjunto de dados
data.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,ID,Betweenness,Closeness,Degree,Page_Rank,Centrality,Cluster_Coefficient,FUNÇÃO,correntista,fake_lover,liderança,não sei,recrutadores_operadores_e_oficiais
0,1,0.289081,0.394990,52,0.039503,0.282880,0.039970,liderança,36,3,0,4,9
1,134,0.001344,0.320814,6,0.004539,0.074397,0.733333,correntista,2,0,2,1,1
2,105,0.025791,0.326433,6,0.005677,0.057502,0.266667,correntista,3,0,2,1,0
3,194,0.000344,0.286313,2,0.002033,0.029641,0.000000,correntista,0,1,1,0,0
4,197,0.000028,0.284722,3,0.002813,0.032340,0.666667,correntista,1,0,1,0,1


In [3]:
data.columns

Index(['ID', 'Betweenness', 'Closeness', 'Degree', 'Page_Rank', 'Centrality',
       'Cluster_Coefficient', 'FUNÇÃO', 'correntista', 'fake_lover',
       'liderança', 'não sei', 'recrutadores_operadores_e_oficiais'],
      dtype='object')

# Organizando as Colunas
Separação das colunas de dados (características/features) e coluna alvo (target ou variável dependente).

In [4]:
from collections import Counter

# A variável X conterá as colunas de 'dados' (variáveis independentes ou características a serem usadas pelo modelo)
feature_cols = ['Betweenness', 'Closeness', 'Degree', 'Page_Rank', 'Centrality',
                'Cluster_Coefficient'
                ]
X = data[feature_cols]

# A variável y conterá a coluna alvo
y = data[TARGET]

# Exibindo o número de instâncias da coluna alvo
print(Counter(y))

#y = y.replace('no', 0)
#y = y.replace('yes', 1)

# Exibindo número de instâncias de acordo com o alvo (não há valores faltantes na versão atual dos dados)
data.groupby(TARGET).count()



Counter({'correntista': 138, 'recrutadores_operadores_e_oficiais': 25, 'fake_lover': 12, 'liderança': 6})


,ID,Betweenness,Closeness,Degree,Page_Rank,Centrality,Cluster_Coefficient,correntista,fake_lover,liderança,não sei,recrutadores_operadores_e_oficiais
FUNÇÃO,,,,,,,,,,,,
correntista,138,138,138,138,138,138,138,138,138,138,138,138
fake_lover,12,12,12,12,12,12,12,12,12,12,12,12
liderança,6,6,6,6,6,6,6,6,6,6,6,6
recrutadores_operadores_e_oficiais,25,25,25,25,25,25,25,25,25,25,25,25


# Divisão do Conjunto de Dados
Não dividiremos o conjunto de dados em treinamento e teste, pois avaliaremos, neste momento, o desempenho dos algoritmos de acordo com os valores da validação cruzada.

In [5]:
from sklearn.model_selection import train_test_split
# Neste momento, não separemos em conjunto de treinamento e teste, pois irão nos interessar apenas os resultados da validação cruzada.
#x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123, stratify=y)
x_train = X
y_train = y

## Carregamento dos Modelos
A seguir são importados os classificadores e funções auxiliares que serão utilizadas.

In [6]:
# Pipeline (execução de vários testes)
from sklearn.pipeline import Pipeline

# Engenharia de Características (projeção, seleção e normalização das características)
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# Modelos (regressores)
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm

# Métricas
#from sklearn import metrics
#from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix, mean_absolute_error

# Hiperparâmetros - Observação
Os melhores valores de hiperparâmetros poderiam ser encontrados usando uma busca em grid (Grid Search). No caso, estamos usando valores arbitrários, padrões ou experimentais.

# Criação dos objetos correspondentes aos classificadores

In [7]:
baseline = DummyClassifier(strategy='most_frequent', random_state = 100, constant = None)
RegLog = LogisticRegression(class_weight='balanced', max_iter=5000, solver='liblinear', penalty='l2', C=1.3225)
MLP = MLPClassifier(activation='relu', max_iter=5000)
GNB = GaussianNB()
RF = RandomForestClassifier()
KNN = KNeighborsClassifier(n_neighbors = 3)
DT = DecisionTreeClassifier (criterion="entropy", max_depth=10)
SVM = svm.SVC()
SVM2 = svm.SVC(kernel='poly', C=1, gamma=1)

## 3. Comparação dos modelos

Agora, vamos criar um pipeline de pipelines para comparar diferentes combinações de modelos e pré-processamentos diferentes. Abaixo, crio uma lista com todas as combinações que quero testar.

OBS.: É importante dar um nome bom para cada uma das etapas; isso vai ajudar a identificar o modelo em questão na fase de comparações.

In [8]:


modelos = [Pipeline(steps=[('dummy', baseline)]),
           Pipeline(steps=[("SemPreprocessamento, RegLog", RegLog)]),
           Pipeline(steps=[('SemPreprocessamento, MLP', MLP)]),
           Pipeline(steps=[('SemPreprocessamento, GNB', GNB)]),
           Pipeline(steps=[('SemPreprocessamento, KNN', KNN)]),
           Pipeline(steps=[('SemPreprocessamento, DT', DT)]),
           Pipeline(steps=[('SemPreprocessamento, RF', RF)]),
           Pipeline(steps=[('SemPreprocessamento, SVM', SVM)]),
           Pipeline(steps=[('SemPreprocessamento, SVM_Poly', SVM2)]),
           Pipeline(steps=[('MinMaxScaler',MinMaxScaler()),('RegLog', RegLog)]),
           Pipeline(steps=[('MinMaxScaler',MinMaxScaler()),('MLP', MLP)]),
           Pipeline(steps=[('MinMaxScaler',MinMaxScaler()),('GNB', GNB)]),
           Pipeline(steps=[('MinMaxScaler',MinMaxScaler()),('KNN', KNN)]),
           Pipeline(steps=[('MinMaxScaler',MinMaxScaler()),('DT', DT)]),
           Pipeline(steps=[('MinMaxScaler',MinMaxScaler()),('RF', RF)]),
           Pipeline(steps=[('MinMaxScaler',MinMaxScaler()),('SVM', SVM)]),
           Pipeline(steps=[('MinMaxScaler',MinMaxScaler()),('SVM_Poly', SVM2)]),
           Pipeline(steps=[('StandardScaler',StandardScaler()),('RegLog', RegLog)]),
           Pipeline(steps=[('StandardScaler',StandardScaler()),('MLP', MLP)]),
           Pipeline(steps=[('StandardScaler',StandardScaler()),('GNB', GNB)]),
           Pipeline(steps=[('StandardScaler',StandardScaler()),('KNN', KNN)]),
           Pipeline(steps=[('StandardScaler',StandardScaler()),('DT', DT)]),
           Pipeline(steps=[('MinMaxScaler',MinMaxScaler()),('RF', RF)]),
           Pipeline(steps=[('StandardScaler',StandardScaler()),('SVM', SVM)]),
           Pipeline(steps=[('StandardScaler',StandardScaler()),('SVM_Poly', SVM2)]),
           Pipeline(steps=[('PCA', TruncatedSVD(DIMENSOES_PCA)),('RegLog', RegLog)]),
           Pipeline(steps=[('PCA', TruncatedSVD(DIMENSOES_PCA)),('MLP', MLP)]),
           Pipeline(steps=[('PCA', TruncatedSVD(DIMENSOES_PCA)),('GNB', GNB)]),
           Pipeline(steps=[('PCA', TruncatedSVD(DIMENSOES_PCA)),('KNN', KNN)]),
           Pipeline(steps=[('PCA', TruncatedSVD(DIMENSOES_PCA)),('DT', DT)]),
           Pipeline(steps=[('PCA', TruncatedSVD(DIMENSOES_PCA)),('RF', RF)]),
           Pipeline(steps=[('PCA', TruncatedSVD(DIMENSOES_PCA)),('SVM', SVM)]),
           Pipeline(steps=[('PCA', TruncatedSVD(DIMENSOES_PCA)),('SVM_Poly', SVM2)]),
           Pipeline(steps=[('KBest', SelectKBest(chi2, k=ATRIBUTOS_KBEST)),('RegLog', RegLog)]),
           Pipeline(steps=[('KBest', SelectKBest(chi2, k=ATRIBUTOS_KBEST)),('MLP', MLP)]),
           Pipeline(steps=[('KBest', SelectKBest(chi2, k=ATRIBUTOS_KBEST)),('GNB', GNB)]),
           Pipeline(steps=[('KBest', SelectKBest(chi2, k=ATRIBUTOS_KBEST)),('KNN', KNN)]),
           Pipeline(steps=[('KBest', SelectKBest(chi2, k=ATRIBUTOS_KBEST)),('DT', DT)]),
           Pipeline(steps=[('KBest', SelectKBest(chi2, k=ATRIBUTOS_KBEST)),('RF', RF)]),
           Pipeline(steps=[('KBest', SelectKBest(chi2, k=ATRIBUTOS_KBEST)),('SVM', SVM)]),
           Pipeline(steps=[('KBest', SelectKBest(chi2, k=ATRIBUTOS_KBEST)),('SVM_Poly', SVM2)])
          ]

# Execução
Os modelos são executados e o desempenho é medido considerando a média dos resultados da validação cruzada.

In [9]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

kf = StratifiedKFold(n_splits=FOLDS, random_state = 123, shuffle = True)

testados = modelos

In [10]:

#for m in testados:
#  print([i for i,j in m.steps], '\tf1_macro\t', round(cross_val_score(m, x_train, y_train, cv=kf, scoring='f1_macro').mean(), 3))
#  print([i for i,j in m.steps], '\taccuracy\t', round(cross_val_score(m, x_train, y_train, cv=kf, scoring='accuracy').mean(), 3))
#  print([i for i,j in m.steps], '\troc_auc\t', round(cross_val_score(m, x_train, y_train, cv=kf, scoring='roc_auc').mean(), 3))


# EXECUÇÃO DE TODOS OS MODELOS PARA TODAS AS VARIÁVEIS ALVO

In [11]:
ALVOS = ['FUNÇÃO']
testados = modelos

In [12]:
for TARGET in ALVOS:
  y = data[TARGET]
  y_train = y
  print(TARGET)
  print(Counter(y), "\n")

FUNÇÃO
Counter({'correntista': 138, 'recrutadores_operadores_e_oficiais': 25, 'fake_lover': 12, 'liderança': 6}) 



In [13]:
#for TARGET in ALVOS:
#  y = data[TARGET]
#  y_train = y
#  print("########## Alvo em análise: ", TARGET, "##########")
#  for m in testados:
#    print([i for i,j in m.steps], '\tf1_macro\t', round(cross_val_score(m, x_train, y_train, cv=kf, scoring='f1_macro').mean(), 3))
#    print([i for i,j in m.steps], '\taccuracy\t', round(cross_val_score(m, x_train, y_train, cv=kf, scoring='accuracy').mean(), 3))
#    print([i for i,j in m.steps], '\troc_auc\t', round(cross_val_score(m, x_train, y_train, cv=kf, scoring='roc_auc').mean(), 3))
#  print("\n\n\n")

In [14]:
import io
import sys
import re

TARGET = ALVOS[0]
y = data[TARGET]
y_train = y

stdout_original = sys.stdout
captura = io.StringIO()
sys.stdout = captura

print("########## Alvo em análise: ", TARGET, "##########")
for m in testados:
  print([i for i,j in m.steps], '\tf1_macro\t', round(cross_val_score(m, x_train, y_train, cv=kf, scoring='f1_macro').mean(), 3))
  print([i for i,j in m.steps], '\taccuracy\t', round(cross_val_score(m, x_train, y_train, cv=kf, scoring='accuracy').mean(), 3))
  #print([i for i,j in m.steps], '\troc_auc\t', round(cross_val_score(m, x_train, y_train, cv=kf, scoring='roc_auc').mean(), 3))
print("\n\n\n")

sys.stdout = stdout_original

linhas = captura.getvalue().splitlines()

dados = []
i = 1
while i < len(linhas) - 1:
    try:
        pipeline_match = re.match(r"(\[.*\])\s+f1_macro\s+([0-9.]+)", linhas[i])
        acc_match = re.match(r"(\[.*\])\s+accuracy\s+([0-9.]+)", linhas[i+1])

        if pipeline_match and acc_match:
            pipeline = pipeline_match.group(1)
            f1 = float(pipeline_match.group(2))
            acc = float(acc_match.group(2))
            dados.append({'Pipeline': pipeline, 'F1 Macro': f1, 'Accuracy': acc})
        i += 2
    except Exception as e:
        print(f"Erro ao processar linha {i}: {e}")
        i += 1

df_resultados = pd.DataFrame(dados)
df_resultados.to_excel('./drive/MyDrive/Análise de redes sociais criminais/resultados_modelos_03.xlsx', index=False)
print("Resultados salvos em 'resultados_modelos_03.xlsx'")

Resultados salvos em 'resultados_modelos_03.xlsx'


In [15]:
df_resultados

,Pipeline,F1 Macro,Accuracy
0,['dummy'],0.216,0.763
1,"['SemPreprocessamento, RegLog']",0.310,0.774
2,"['SemPreprocessamento, MLP']",0.377,0.807
3,"['SemPreprocessamento, GNB']",0.434,0.719
4,"['SemPreprocessamento, KNN']",0.364,0.774
5,"['SemPreprocessamento, DT']",0.379,0.702
6,"['SemPreprocessamento, RF']",0.392,0.779
7,"['SemPreprocessamento, SVM']",0.313,0.791
8,"['SemPreprocessamento, SVM_Poly']",0.366,0.779
9,"['MinMaxScaler', 'RegLog']",0.298,0.785
